In [2]:
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency,ttest_ind,mannwhitneyu,kstest
from sklearn.metrics import confusion_matrix
import pandas as pd
import json

In [3]:
import pandas as pd 
data = pd.read_excel('./data/data_gt_60_new0901.xlsx')

In [11]:
# 定量分析的清单:数值类型的数据
quantitative_analysis = ["心率","舒张压","收缩压","年龄", "LDL-C检测值", "HDL-C检测值",
                         "TC检测值", "空腹血糖检测值","最高值肌酐检测值","首次血红蛋白检测值",
                        "CK-MB最高值检测值","TG检测值", "肌钙蛋白I最高值检测值","肌钙蛋白T最高值检测值",
                        "肌红蛋白最高值检测值","糖化血红蛋白检测值","BNP检测值","NT-proBNP检测值","LVEF","乳酸脱氢酶最高值检测值.2",
                        "TyG指数", "TG/HDL"]



# 定性分析的清单: 标签类型的数据 男女 是否
qualitative_analysis = ["性别","戒烟：无","吸烟史","饮酒史","既往心梗","脑血管病史",
                        '合并高血压','合并糖尿病',"既往心梗","心衰史","既往CABG","既往PCI",
                        "本次住院是否进行CABG治疗","室壁瘤.1","是否行PCI",
                        "乳头肌功能不全或断裂","室壁破裂","室间隔穿孔","附壁血栓","室壁瘤","并发症其他",
                        "冠状动脉支架植入术后再狭窄","冠状动脉支架内血栓形成","急性亚急性支架血栓形成","急性再发心肌梗死","再梗死","消化道出血.1","心脏瓣膜病",
                        "其他出血事件",
                        "入院诊断","首次心功能分级","出院主要诊断","LM_LAD_LCX_RCA" # 不是二值变量
                       ] 
mace_df = data[data['mace']==True]
non_mace_df = data[data['mace']==False]

In [5]:
save_list = []

def generate_array(demo_df, col_name):
    group_stat = demo_df.value_counts([col_name,'mace']).reset_index().rename({0:"count"},axis=1)
    group_df = group_stat.pivot(index=col_name,columns='mace',values='count')
    # print(group_df)
    return np.array(group_df)

for col_name in qualitative_analysis: # 对定性分析的数据进行 卡方检验
    # 输出内容： mace 组的值 非mace组的各值
    print(col_name)
    # if len(data[col_name].dropna()) < 1134:
    #     # print(f"{col_name}")
    #     print(data[col_name].value_counts())
    cur_mace_info = mace_df[col_name].value_counts().to_dict()
    cur_non_mace_info = non_mace_df[col_name].value_counts().to_dict()
    
    # print(data[col_name].value_counts())
    
    cal_mat = generate_array(data,col_name)
    chi2_res = chi2_contingency(cal_mat,correction=False)
    # print()
    save_list.append([col_name,
                      json.dumps(cur_mace_info,ensure_ascii=False), 
                     json.dumps(cur_non_mace_info,ensure_ascii=False),
                      round(chi2_res.statistic,4),
                     round(chi2_res.pvalue,4)])
save_df = pd.DataFrame(save_list,columns=["指标","mace","非mace","统计值","p-value"])

性别
戒烟：无
吸烟史
饮酒史
既往心梗
脑血管病史
合并高血压
合并糖尿病
既往心梗
心衰史
既往CABG
既往PCI
本次住院是否进行CABG治疗
室壁瘤.1
是否行PCI
乳头肌功能不全或断裂
室壁破裂
室间隔穿孔
附壁血栓
室壁瘤
并发症其他
冠状动脉支架植入术后再狭窄
冠状动脉支架内血栓形成
急性亚急性支架血栓形成
急性再发心肌梗死
再梗死
消化道出血.1
心脏瓣膜病
其他出血事件
入院诊断
首次心功能分级
出院主要诊断
LM_LAD_LCX_RCA


In [11]:
save_df

,指标,mace,非mace,统计值,p-value
0,性别,"{""男性"": 46, ""女性"": 13}","{""男性"": 310, ""女性"": 91}",0.0128,0.9100
1,戒烟：无,"{""否"": 42, ""是"": 17}","{""否"": 254, ""是"": 147}",1.3797,0.2402
2,吸烟史,"{""否"": 38, ""是"": 21}","{""否"": 247, ""是"": 154}",0.1724,0.6780
3,饮酒史,"{""否"": 46, ""是"": 13}","{""否"": 328, ""是"": 73}",0.4962,0.4812
4,既往心梗,"{""否"": 54, ""是"": 5}","{""否"": 378, ""是"": 23}",0.6749,0.4113
5,脑血管病史,"{""否"": 49, ""是"": 10}","{""否"": 377, ""是"": 24}",9.0326,0.0027
6,合并高血压,"{""true"": 43, ""false"": 16}","{""true"": 258, ""false"": 143}",1.6593,0.1977
7,合并糖尿病,"{""false"": 30, ""true"": 29}","{""false"": 268, ""true"": 133}",5.7607,0.0164
8,既往心梗,"{""否"": 54, ""是"": 5}","{""否"": 378, ""是"": 23}",0.6749,0.4113
9,心衰史,"{""否"": 58, ""是"": 1}","{""否"": 400, ""是"": 1}",2.4827,0.1151


In [12]:
mace_total = 46 + 13
non_mace_total = 310 + 91
mace_ratio, non_mace_ratio = [], []
for idx, col in save_df.iterrows():
    mace_cont = json.loads(col['mace'])
    non_mace_cont = json.loads(col['非mace'])
    if "是" in mace_cont:
        mace_cnt = mace_cont['是']
        non_mace_cnt = non_mace_cont.get('是',0)
    elif "true" in mace_cont:
        mace_cnt = mace_cont['true']
        non_mace_cnt = non_mace_cont['true']
    else:
        label = list(mace_cont.keys())[0]
        mace_cnt  = mace_cont[label]
        non_mace_cnt  = non_mace_cont[label]
    mace_ratio.append(f"{round(mace_cnt * 100 / mace_total, 2)}%")
    non_mace_ratio.append(f"{round(non_mace_cnt * 100 / non_mace_total, 2)}%")   
save_df['mace比例'] = mace_ratio
save_df['非mace比例'] = non_mace_ratio
dx_df = save_df # 定性分析结果
del save_df

In [13]:
dx_df

,指标,mace,非mace,统计值,p-value,mace比例,非mace比例
0,性别,"{""男性"": 46, ""女性"": 13}","{""男性"": 310, ""女性"": 91}",0.0128,0.9100,77.97%,77.31%
1,戒烟：无,"{""否"": 42, ""是"": 17}","{""否"": 254, ""是"": 147}",1.3797,0.2402,28.81%,36.66%
2,吸烟史,"{""否"": 38, ""是"": 21}","{""否"": 247, ""是"": 154}",0.1724,0.6780,35.59%,38.4%
3,饮酒史,"{""否"": 46, ""是"": 13}","{""否"": 328, ""是"": 73}",0.4962,0.4812,22.03%,18.2%
4,既往心梗,"{""否"": 54, ""是"": 5}","{""否"": 378, ""是"": 23}",0.6749,0.4113,8.47%,5.74%
5,脑血管病史,"{""否"": 49, ""是"": 10}","{""否"": 377, ""是"": 24}",9.0326,0.0027,16.95%,5.99%
6,合并高血压,"{""true"": 43, ""false"": 16}","{""true"": 258, ""false"": 143}",1.6593,0.1977,72.88%,64.34%
7,合并糖尿病,"{""false"": 30, ""true"": 29}","{""false"": 268, ""true"": 133}",5.7607,0.0164,49.15%,33.17%
8,既往心梗,"{""否"": 54, ""是"": 5}","{""否"": 378, ""是"": 23}",0.6749,0.4113,8.47%,5.74%
9,心衰史,"{""否"": 58, ""是"": 1}","{""否"": 400, ""是"": 1}",2.4827,0.1151,1.69%,0.25%


In [13]:
dx_df.to_excel('./data/定性分析结果-0827.xlsx',index=None)

In [12]:
save_list = []
for col_name in quantitative_analysis: # 对定量分析的数据进行 先判断是否符合正正态分布 满足t检验 不满足 u检验
    # 输出内容： mace 组的值 非mace组的各值
    data_mean,data_std = np.mean(data[col_name]), np.std(data[col_name]) 
    norm_res = stats.kstest(data[col_name], 'norm', (data_mean, data_std))
    if norm_res.pvalue > 0.05: # 大于0.05就是正态分布
        is_norm = True
    else:
        is_norm = False
    
    # print(col_name)
    # 填充空值
    mace_df[col_name] = mace_df[col_name].fillna(value=mace_df[col_name].mean())
    non_mace_df[col_name] = non_mace_df[col_name].fillna(value=non_mace_df[col_name].mean())
    
    
    # 原有执行流程
    cur_mace_arr = mace_df[col_name].dropna()
    mace_mean, mace_std = np.mean(cur_mace_arr), np.std(cur_mace_arr)
    cur_non_mace_arr = non_mace_df[col_name].dropna()
    # if len(cur_mace_arr) + len(cur_non_mace_arr) != 1134:
    #     print(f"{col_name}\t {len(cur_mace_arr) + len(cur_non_mace_arr)}")
    no_mace_mean, no_mace_std = np.mean(cur_non_mace_arr), np.std(cur_non_mace_arr)
    
    # 对数据进行是否正态分布的验证
    statistic_t, p_value_t = ttest_ind(cur_mace_arr, cur_non_mace_arr)

    statistic_u, p_value_u = mannwhitneyu(cur_mace_arr,cur_non_mace_arr,alternative='two-sided')
    mace_quater_arr1, mace_quater_arr2,mace_quater_arr3 = np.percentile(cur_mace_arr, [25, 50, 75]).tolist()
    non_quater_arr1, non_quater_arr2,non_quater_arr3 = np.percentile(cur_non_mace_arr, [25, 50, 75]).tolist()
    
    # print(data[col_name].value_counts())
    # cal_mat = generate_array(data,col_name)
    # chi2_res = chi2_contingency(cal_mat)
    # print()
    save_list.append([col_name,
        f"{round(mace_mean,2)}±{round(mace_std,2)}",
        f"{round(no_mace_mean,2)}±{round(no_mace_std,2)}",
          round(statistic_t,4),
          f"{round(mace_quater_arr2,2)}({round(mace_quater_arr1,2)},{round(mace_quater_arr3,2)})",
          f"{round(non_quater_arr2,2)}({round(non_quater_arr1,2)},{round(non_quater_arr3,2)})",
         round(p_value_t,4),
         round(statistic_u,4),
         round(p_value_u,4),
         is_norm ])
save_df = pd.DataFrame(save_list,columns=["指标","mace","非mace","t统计值","mace四分位数","非mace四分位数", "p-value-t", "u统计值", "p-value-u", "是否正态分布"])

/var/folders/6y/2k_kxtyd07v79br33f6gvjn00000gn/T/ipykernel_1252/1646894356.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mace_df[col_name] = mace_df[col_name].fillna(value=mace_df[col_name].mean())
/var/folders/6y/2k_kxtyd07v79br33f6gvjn00000gn/T/ipykernel_1252/1646894356.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mace_df[col_name] = non_mace_df[col_name].fillna(value=non_mace_df[col_name].mean())


In [15]:
dl_df = save_df # 定量分析结果
del save_df
dl_df.to_excel('./data/定量分析结果-0827.xlsx',index=None)

In [13]:
save_df

,指标,mace,非mace,t统计值,mace四分位数,非mace四分位数,p-value-t,u统计值,p-value-u,是否正态分布
0,心率,79.15±13.53,73.35±13.84,3.0112,"80.0(70.0,88.5)","71.0(65.0,80.0)",0.0027,14924.5,0.0012,False
1,舒张压,70.97±11.01,72.59±10.81,-1.0746,"70.0(62.5,80.0)","70.0(65.0,80.0)",0.2831,11160.5,0.4813,False
2,收缩压,118.02±20.77,121.31±18.28,-1.2644,"116.0(104.0,124.5)","120.0(110.0,132.0)",0.2067,10366.0,0.1245,False
3,年龄,68.3±6.27,67.65±5.61,0.8157,"66.0(63.97,72.03)","66.77(63.0,71.0)",0.4151,12434.5,0.5260,False
4,LDL-C检测值,3.09±0.97,2.59±0.91,3.8444,"3.04(2.42,3.67)","2.48(1.98,3.15)",0.0001,15378.0,0.0002,True
5,HDL-C检测值,1.25±0.31,0.94±0.29,7.6594,"1.28(1.05,1.44)","0.9(0.74,1.14)",0.0000,18293.5,0.0000,True
6,TC检测值,4.41±1.13,4.43±1.09,-0.1273,"4.11(3.67,5.13)","4.36(3.7,5.06)",0.8988,11461.5,0.6999,True
7,空腹血糖检测值,8.57±3.27,7.37±2.92,2.8761,"7.46(6.1,9.45)","6.57(5.41,8.6)",0.0042,14909.5,0.0012,False
8,最高值肌酐检测值,110.09±97.72,90.91±39.06,2.7158,"84.4(68.5,103.34)","80.1(67.6,99.34)",0.0069,12602.0,0.4181,False
9,首次血红蛋白检测值,139.92±18.92,141.0±16.18,-0.4692,"142.0(131.5,149.0)","142.0(132.0,151.0)",0.6392,11486.5,0.7193,True
